# crspm

> Retrieve and process data from WRDS CRSP Monthly Stock File

Since this is a proprietary dataset, in the documentation below, I can not show any of the data that is being retrieved/generated (will show only column names).

In [ ]:
#| default_exp wrds.crspm

In [ ]:
#|exports
from __future__ import annotations
from pathlib import Path
from typing import List
import os

import pandas as pd
import numpy as np

import pandasmore as pdm
from finsets.wrds import wrds_api
from finsets import RESOURCES

In [ ]:
#| exports
PROVIDER = 'Wharton Research Data Services (WRDS)'
URL = 'https://wrds-www.wharton.upenn.edu/pages/get-data/center-research-security-prices-crsp/annual-update/stock-security-files/monthly-stock-file/'
LIBRARY = 'crsp'
TABLE = 'msf'
NAMES_TABLE = 'msenames'
DELIST_TABLE = 'msedelist'
FREQ = 'M'
MIN_YEAR = 1925
MAX_YEAR = None
ENTITY_ID_IN_RAW_DSET = 'permno'
ENTITY_ID_IN_CLEAN_DSET = 'permno'
TIME_VAR_IN_RAW_DSET = 'date'
TIME_VAR_IN_CLEAN_DSET = 'Mdate'
LABELS_FILE = RESOURCES/'crspm_variable_descriptions.csv'

In [ ]:
#| export
def raw_metadata() -> pd.DataFrame:
    "Collects metadata from WRDS `{LIBRARY}.{TABLE}` and `{LIBRARY}.{NAMES_TABLE}` tables and merges it with variable labels from LABELS_FILE"

    try:
        db = wrds_api.Connection()
        msf = db.describe_table(LIBRARY,TABLE)
        msf_rows = db.get_row_count(LIBRARY,TABLE)
        mse = db.describe_table(LIBRARY,NAMES_TABLE)
        mse_rows = db.get_row_count(LIBRARY,NAMES_TABLE)
    finally:
        db.close()
        
    msf_meta = msf[['name','type']].copy()
    msf_meta['nr_rows'] = msf_rows
    msf_meta['wrds_library'] = LIBRARY
    msf_meta['wrds_table'] = TABLE

    mse_meta = mse[['name','type']].copy()
    mse_meta['nr_rows'] = mse_rows
    mse_meta['wrds_library'] = LIBRARY
    mse_meta['wrds_table'] = NAMES_TABLE

    # Get variable labels from LABELS_FILE
    df = pd.read_csv(LABELS_FILE)
    df['Variable Label'] = df.apply(lambda row: row['Description'].replace(row['Variable Name'].strip()+' -- ', ''), axis=1)
    df['Variable Label'] = df.apply(lambda row: row['Variable Label'].replace( '(' + row['Variable Name'].strip() + ')', ''), axis=1)
    df['Variable Name'] = df['Variable Name'].str.strip().str.lower()
    df = df[['Variable Name', 'Variable Label']].copy()
    df.columns = ['name','label']

    # Merge metadata and variable labels and clean up a bit
    crsp_meta = (pd.concat([msf_meta, mse_meta],axis=0, ignore_index=True)
                .merge(df, how='left', on='name'))
    crsp_meta['output_of'] = 'wrds.crspm.download'
    crsp_meta = pdm.order_columns(crsp_meta,these_first=['name','label','output_of'])
    for v in list(crsp_meta.columns):
        crsp_meta[v] = crsp_meta[v].astype('string')
    
    return crsp_meta

In [ ]:
#| eval: false
raw_metadata().head()

Loading library list...
Done
Approximately 4922867 rows in crsp.msf.
Approximately 111623 rows in crsp.msenames.


,name,label,output_of,type,nr_rows,wrds_library,wrds_table
0,cusip,Cusip,wrds.crspm.download,VARCHAR(8),4922867,crsp,msf
1,permno,<NA>,wrds.crspm.download,DOUBLE_PRECISION,4922867,crsp,msf
2,permco,CRSP Permanent Company Number,wrds.crspm.download,DOUBLE_PRECISION,4922867,crsp,msf
3,issuno,Nasdaq Issue Number,wrds.crspm.download,DOUBLE_PRECISION,4922867,crsp,msf
4,hexcd,Header Exchange Code,wrds.crspm.download,DOUBLE_PRECISION,4922867,crsp,msf


In [ ]:
#| export
def default_raw_vars():
    """Default variables used in `download` if none are specified."""
    
    return ['permno','permco','date',
            'ret', 'retx', 'shrout', 'prc', 
            'shrcd', 'exchcd','siccd','ticker','cusip','ncusip']            

In [ ]:
print(default_raw_vars())

['permno', 'permco', 'date', 'ret', 'retx', 'shrout', 'prc', 'shrcd', 'exchcd', 'siccd', 'ticker', 'cusip', 'ncusip']


In [ ]:
vars = ['a', 'a']
list(set(vars))

['a']

In [ ]:
#| export
def parse_varlist(vars: List[str]=None,
                  required_vars = ['permno','permco','date'],
                  add_delist_adj_ret: str=None
                  ) -> str:
    """Figure out which `vars` come from the `{LIBRARY}.{TABLE}` table and which come from the `{LIBRARY}.{NAMES_TABLE}` table and add a. and b. prefixes"""

    if vars is None: vars = default_raw_vars()
    adj_vars = ['ret','exchcd','dlret','dlstcd','dlstdt'] if add_delist_adj_ret else []
    req_vars = required_vars + [x for x in adj_vars if x not in required_vars]
    all_vars =  req_vars + [x for x in list(set(vars)) if x not in req_vars]

    try:
        db = wrds_api.Connection()
        all_msf_vars = list(db.describe_table(LIBRARY,TABLE).name)
        all_mse_vars = list(db.describe_table(LIBRARY,NAMES_TABLE).name)
        all_delist_vars = list(db.describe_table(LIBRARY,DELIST_TABLE).name) if add_delist_adj_ret else []
        my_msf_vars = [f'a.{x}' for x in all_vars if x in all_msf_vars]
        my_mse_vars = [f'b.{x}' for x in all_vars if (x in all_mse_vars) 
                                                        and (x not in all_msf_vars)]
        my_delist_vars = [f'c.{x}' for x in all_vars if (x in all_delist_vars) 
                                                        and (x not in all_mse_vars) and (x not in all_msf_vars)]
        varlist_string = ','.join(my_msf_vars + my_mse_vars + my_delist_vars)
    finally:
        db.close()
        
    return varlist_string

In [ ]:
#| export
def delist_adj_ret(df: pd.DataFrame, # Requires `ret`,`exchcd`, `dlret`, and `dlstcd` variables
                       adj_ret_var: str
                       ) -> pd.DataFrame:
    """Adjust for delisting returns using Shumway and Warther (1999) and Johnson and Zhao (2007)"""

    df['npdelist'] = (df['dlstcd']==500) | df['dlstcd'].between(520,584)
    df['dlret'] = np.where(df.dlret.isna() & df.npdelist & df.exchcd.isin([1,2]), -0.35, df.dlret)
    df['dlret'] = np.where(df.dlret.isna() & df.npdelist & df.exchcd.isin([3]), -0.55, df.dlret)
    df['dlret'] = np.where(df.dlret.notna() & df.dlret < -1, -1, df.dlret)
    df['dlret'] = df.dlret.fillna(0)

    df[adj_ret_var] = (1 + df.ret) * (1 + df.dlret) - 1
    df[adj_ret_var] = np.where(df[adj_ret_var].isna() & (df.dlret!=0), df.dlret, df[adj_ret_var])
    df = df.drop('npdelist', axis=1) 
    return df

In [ ]:
#| export
def download(vars: List[str]=None, # If None, downloads `default_raw_vars`; `permno`, `permco`, `date` are added by default
             obs_limit: int=None,  #Number of rows to download. If None, full dataset will be downloaded             
             start_date: str=None,          # Start date in MM/DD/YYYY format
             end_date: str=None,            # End date in MM/DD/YYYY format  
             add_delist_adj_ret: bool=False, # Whether to calculate delisting-adjusted returns 
             adj_ret_var: str='ret_adj'     # What to call the returns adjusted for delisting bias
             ) -> pd.DataFrame:
    """Downloads `vars` from `start_date` to `end_date` from WRDS {LIBRARY}.{TABLE} and {LIBRARY}.{NAMES_TABLE} datasets. 
        Creates `ret_adj` for delisting based on Shumway and Warther (1999) and Johnson and Zhao (2007)"""

    varlist_string = parse_varlist(vars, add_delist_adj_ret=add_delist_adj_ret)
    sql_string = f"""SELECT {varlist_string}
                        FROM {LIBRARY}.{TABLE} AS a 
                        LEFT JOIN {LIBRARY}.{NAMES_TABLE} AS b
                            ON a.permno=b.permno AND b.namedt<=a.date AND a.date<=b.nameendt 
                    """ 
    if add_delist_adj_ret: sql_string +=  f""" LEFT JOIN {LIBRARY}.{DELIST_TABLE} as c
                            ON a.permno=c.permno AND date_trunc('month', a.date) = date_trunc('month', c.dlstdt)
                            """
    sql_string += "WHERE 1=1 "
    if start_date is not None: sql_string += r" AND date >= %(start_date)s"
    if end_date is not None: sql_string += r" AND date <= %(end_date)s"
    if obs_limit is not None: sql_string += r" LIMIT %(obs_limit)s"

    df = wrds_api.download(sql_string,
                             params={'start_date':start_date, 'end_date':end_date, 'obs_limit':obs_limit})
    
    if add_delist_adj_ret: df = delist_adj_ret(df, adj_ret_var)
    return df 

In [ ]:
#| eval: false
raw = download(vars = ['ret'], start_date='01/01/2021', end_date='01/01/2022', obs_limit=100)

Loading library list...
Done
Approximately 4922867 rows in crsp.msf.
Approximately 111623 rows in crsp.msenames.
Loading library list...
Done


Note that we requested only the `prc` variable, but because the `add_delist_adj_ret` parameter defaults to `True`, we get all the variables needed to calculate delisting-adjusted returns:

In [ ]:
#| eval: false
raw.head()

,permno,permco,date,ret
0,10026.0,7976.0,2021-01-29,-0.017442
1,10028.0,7978.0,2021-01-29,0.203846
2,10032.0,7980.0,2021-01-29,-0.016494
3,10044.0,7992.0,2021-01-29,0.071605
4,10051.0,7999.0,2021-01-29,-0.068213


In [ ]:
#| eval: false
raw = download(vars = ['ret'],add_delist_adj_ret=True, obs_limit=100)

Loading library list...
Done
Approximately 4922867 rows in crsp.msf.
Approximately 111623 rows in crsp.msenames.
Approximately 36945 rows in crsp.msedelist.
Loading library list...
Done


Note that we requested only the `prc` variable, but because the `add_delist_adj_ret` parameter defaults to `True`, we get all the variables needed to calculate delisting-adjusted returns:

In [ ]:
#| eval: false
raw.head()

,permno,permco,date,ret,exchcd,dlret,dlstcd,dlstdt,ret_adj
0,10000.0,7952.0,1985-12-31,NaN,NaN,0.0,NaN,None,NaN
1,10000.0,7952.0,1986-01-31,NaN,3.0,0.0,NaN,None,NaN
2,10000.0,7952.0,1986-02-28,-0.257143,3.0,0.0,NaN,None,-0.257143
3,10000.0,7952.0,1986-03-31,0.365385,3.0,0.0,NaN,None,0.365385
4,10000.0,7952.0,1986-04-30,-0.098592,3.0,0.0,NaN,None,-0.098592


In [ ]:
#| export
def clean(df: pd.DataFrame=None,        # If None, downloads `vars` using `download` function; else, must contain `permno` and `date` columns
          vars: List[str]=None,         # If None, downloads `default_raw_vars`
          obs_limit: int=None, #Number of rows to download. If None, full dataset will be downloaded
          start_date: str="01/01/1900", # Start date in MM/DD/YYYY format
          end_date: str=None,           # End date. Default is current date          
          clean_kwargs: dict={},        # Params to pass to `pdm.setup_panel` other than `panel_ids`, `time_var`, and `freq`
          ) -> pd.DataFrame:
    """Applies `pandasmore.setup_panel` to `df`. If `df` is None, downloads `vars` using `download` function."""

    if df is None: df = download(vars=vars, obs_limit=obs_limit, start_date=start_date, end_date=end_date)
    df = pdm.setup_panel(df, panel_ids=ENTITY_ID_IN_RAW_DSET, time_var=TIME_VAR_IN_RAW_DSET, freq=FREQ, **clean_kwargs)
    return df 

In [ ]:
#| eval: false
df = clean(df=raw)

In [ ]:
#| eval: false
df.head()

date     dtdate  permco       ret  exchcd  dlret  \
permno Mdate                                                             
10000  1985-12  1985-12-31 1985-12-31  7952.0       NaN     NaN    0.0   
       1986-01  1986-01-31 1986-01-31  7952.0       NaN     3.0    0.0   
       1986-02  1986-02-28 1986-02-28  7952.0 -0.257143     3.0    0.0   
       1986-03  1986-03-31 1986-03-31  7952.0  0.365385     3.0    0.0   
       1986-04  1986-04-30 1986-04-30  7952.0 -0.098592     3.0    0.0   

                dlstcd dlstdt   ret_adj  
permno Mdate                             
10000  1985-12     NaN   None       NaN  
       1986-01     NaN   None       NaN  
       1986-02     NaN   None -0.257143  
       1986-03     NaN   None  0.365385  
       1986-04     NaN   None -0.098592

We can download a small sample of the dataset and clean it in one step:

In [ ]:
#| eval: false
df = clean(obs_limit=100, vars=['ret','shrcd','exchcd'], start_date='01/01/2020', end_date='12/31/2020')

Loading library list...
Done
Approximately 4922867 rows in crsp.msf.
Approximately 111623 rows in crsp.msenames.
Loading library list...
Done


In [ ]:
#| eval: false
df.head()

,,date,dtdate,permco,ret,exchcd,shrcd
permno,Mdate,,,,,,
10026,2020-01,2020-01-31,2020-01-31,7976.0,-0.100016,3.0,11.0
10028,2020-01,2020-01-31,2020-01-31,7978.0,0.607407,2.0,11.0
10032,2020-01,2020-01-31,2020-01-31,7980.0,-0.075643,3.0,11.0
10044,2020-01,2020-01-31,2020-01-31,7992.0,-0.098592,3.0,11.0
10051,2020-01,2020-01-31,2020-01-31,7999.0,-0.115176,1.0,11.0


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()